In [ ]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import psycopg2

In [ ]:
def retrieve_data(cardnames, setnames, foils):
    conn = psycopg2.connect(host="localhost", port=5432, database="", user="", password="")
    cur = conn.cursor()

    for cardname, setname, foil in zip(cardnames, setnames, foils):
        SQL = 'SELECT uuid, tcgplayerproductid FROM cardinfo WHERE ckname=\'' + cardname + '\' AND ckset=\'' + setname + '\';'
        id_df = pd.read_sql_query(SQL, conn)

        if len(id_df) == 0:
            print('Data not found')
            exit()
        elif len(id_df) >= 2:
            print('Too many cards found')
            exit()

        uuid = id_df['uuid'][0]
        prod_id = id_df['tcgplayerProductId'][0]

        datasets = []

        SQL = 'SELECT tcgdate, lowprice, marketprice FROM TCGpricedata WHERE tcgplayerProductId=' + str(
            prod_id) + ' AND foil=' + str(foil) + ';'
        tcg_price_df = pd.read_sql(SQL, cnxn).rename(columns={"tcgdate": "date"})
        datasets.append(tcg_price_df)

        SQL = 'SELECT mmdate, mmbuyprice FROM MMbuydata WHERE uuid=\'' + uuid + '\';'
        mm_buy_df = pd.read_sql(SQL, cnxn).rename(columns={"mmdate": "date"})
        datasets.append(mm_buy_df)

        SQL = 'SELECT csidate, csibuyprice FROM CSIbuydata WHERE uuid=\'' + uuid + '\';'
        csi_buy_df = pd.read_sql(SQL, cnxn).rename(columns={"csidate": "date"})
        datasets.append(csi_buy_df)

        SQL = 'SELECT ckdate, ckbuyprice FROM CKbuydata WHERE uuid=\'' + uuid + '\';'
        ck_buy_df = pd.read_sql(SQL, cnxn).rename(columns={"ckdate": "date"})
        datasets.append(ck_buy_df)

        SQL = 'SELECT [95date], [95buyprice] FROM 95buydata WHERE uuid=\'' + uuid + '\';'
        ninety_buy_df = pd.read_sql(SQL, cnxn).rename(columns={"95date": "date"})
        datasets.append(ninety_buy_df)

        for df in datasets:
            df.sort_values('date', inplace=True)
        # print(pd.to_datetime(ninety_buy_df['95date']))
        # print(ninety_buy_df['95date'][0].date == dt.datetime(year=2020, month=8, day=4))

        fig, ax = plt.subplots()

        for df in datasets:
            # todo fill in the missing values with 0
            try:
                ax.plot(df.iloc[:, 0], df.iloc[:, 1], label=df.columns[1] + ': ' + str(df.iloc[-1, 1]), linewidth=3, alpha=.7)
                if len(df.columns) == 3:
                    ax.plot(df.iloc[:, 0], df.iloc[:, 2], label=df.columns[2] + ': ' + str(df.iloc[-1, 2]), linewidth=3,
                            alpha=.7)
            except Exception:
                pass

        fig.autofmt_xdate()
        ax.set_xlim(dt.datetime(year=2020, month=8, day=4))
        ax.set_ylim(0)
        formatter = ticker.FormatStrFormatter('$%1.2f')
        ax.yaxis.set_major_formatter(formatter)
        ax.grid()

        if foil:
            f = 'Foil'
        else:
            f = 'Nonfoil'
        ax.set_title(f'{cardname}, {setname}, {f}')
        ax.set_ylabel('price (USD)')
        ax.set_xlabel('date')
        ax.legend()
        ax.set_ylim(top=ax.get_ylim()[1]*1.1)

        plt.show()